In [1]:
from google.colab import files
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/Bot_sides')

Mounted at /content/drive


In [2]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import pickle
import numpy as np

from keras.models import load_model
model = load_model('chatbot_model.h5')
import json
import random
intents = json.loads(open('intents.json').read())
words = pickle.load(open('words.pkl','rb'))
classes = pickle.load(open('classes.pkl','rb'))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [11]:
def clean_up_sentence(sentence):
    # tokenize the pattern - split words into array
    sentence_words = nltk.word_tokenize(sentence)
    # stem each word - create short form for word
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words
# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence

def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0]*len(words) 
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s: 
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return(np.array(bag))

def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words,show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i,r] for i,r in enumerate(res)]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list

In [12]:
def getResponse(ints, intents_json,index):
    tag = ints[index]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if(i['tag']== tag):
            result = random.choice(i['responses'])
            break
    return tag,result

def chatbot_response(current_tags,text):
    ints = predict_class(text, model)
    l = len(ints)
    index = 0
    for i in range(l):
        if ints[i]['intent'] not in current_tags:
            index=i
            break
    new_tag, res = getResponse(ints, intents,index)
    return new_tag,res

In [14]:
def chat():
    tag_dict = {"start","feel","sleep","work","physical health","confidence","suicidal thoughts"}
    current_tags = set()
    print("Start talking with the bot (type quit to stop)!")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        new_tag,result = chatbot_response(current_tags,inp)
        current_tags.add(new_tag)
        print(result)

chat()

Start talking with the bot (type quit to stop)!
You: hi
Hi, nice to see you!!!
You: what do you do
I am Maya. I want to talk to you to learn more about your mental state so that I can help you out. I want you to feel uplifted and good. It will be great for me if you are a little detailed while talking. I am being consistently updated and at some point, I will hopefully be able to extract meaningful information from even an undetailed conversation just like a human being,  but for the time being thanks for your consideration :) If you want to allow me to know your mental state, type “help” 
You: start
How have you been feeling recently? This is like your overall feeling about yourself recently, because I will ask specific questions later
You: I have been feeling well recently
I see. Tell me about your sleeping habits over the past X months. Have you noticed any changes or Insomnia? Difficulty sleeping? Restlessness or early morning wakefulness or sleeping too much? How about the quality